---

### 🏗 Project: `crudproject`

App: `students`

We will build a simple student record system to:

* Create student entries
* View all students
* Update student info
* Delete students

---

### ✅ STEP 1: Create the Django Project and App

```bash
django-admin startproject crudproject
cd crudproject
python manage.py startapp students
```

---

### ✅ STEP 2: Register the App

In `crudproject/settings.py`:

```python
INSTALLED_APPS = [
    ...,
    'students',
]
```

---

### ✅ STEP 3: Create the Model

In `students/models.py`:

```python
from django.db import models

class Student(models.Model):
    name = models.CharField(max_length=100)
    email = models.EmailField()
    course = models.CharField(max_length=50)

    def __str__(self):
        return self.name
```

---

### ✅ STEP 4: Migrate the Database

```bash
python manage.py makemigrations
python manage.py migrate
```

---

### ✅ STEP 5: Create the Form

In `students/forms.py`:

```python
from django import forms
from .models import Student

class StudentForm(forms.ModelForm):
    class Meta:
        model = Student
        fields = ['name', 'email', 'course']
```

---

### ✅ STEP 6: Create Views

In `students/views.py`:

```python
from django.shortcuts import render, redirect, get_object_or_404
from .models import Student
from .forms import StudentForm

def student_list(request):
    students = Student.objects.all()
    return render(request, 'students/student_list.html', {'students': students})

def student_create(request):
    if request.method == 'POST':
        form = StudentForm(request.POST)
        if form.is_valid():
            form.save()
            return redirect('student_list')
    else:
        form = StudentForm()
    return render(request, 'students/student_form.html', {'form': form})

def student_update(request, pk):
    student = get_object_or_404(Student, pk=pk)
    form = StudentForm(request.POST or None, instance=student)
    if form.is_valid():
        form.save()
        return redirect('student_list')
    return render(request, 'students/student_form.html', {'form': form})

def student_delete(request, pk):
    student = get_object_or_404(Student, pk=pk)
    if request.method == 'POST':
        student.delete()
        return redirect('student_list')
    return render(request, 'students/student_confirm_delete.html', {'student': student})
```

---

### ✅ STEP 7: Create URLs

In `students/urls.py`:

```python
from django.urls import path
from . import views

urlpatterns = [
    path('', views.student_list, name='student_list'),
    path('create/', views.student_create, name='student_create'),
    path('update/<int:pk>/', views.student_update, name='student_update'),
    path('delete/<int:pk>/', views.student_delete, name='student_delete'),
]
```

In `crudproject/urls.py`:

```python
from django.contrib import admin
from django.urls import path, include

urlpatterns = [
    path('admin/', admin.site.urls),
    path('', include('students.urls')),
]
```

---

### ✅ STEP 8: Create Templates

Create the folder: `students/templates/students/`

### 🔸 `student_list.html`

```html
<!DOCTYPE html>
<html>
<head>
    <title>Student List</title>
</head>
<body>
    <h1>Student Records</h1>
    <a href="{% url 'student_create' %}">Add Student</a>
    <ul>
        {% for student in students %}
        <li>
            {{ student.name }} – {{ student.course }}
            <a href="{% url 'student_update' student.pk %}">Edit</a>
            <a href="{% url 'student_delete' student.pk %}">Delete</a>
        </li>
        {% empty %}
        <li>No students found.</li>
        {% endfor %}
    </ul>
</body>
</html>
```

---

### 🔸 `student_form.html`

```html
<!DOCTYPE html>
<html>
<head>
    <title>Add/Edit Student</title>
</head>
<body>
    <h1>{{ form.instance.pk|yesno:"Edit Student,Add Student" }}</h1>
    <form method="post">
        {% csrf_token %}
        {{ form.as_p }}
        <button type="submit">Save</button>
    </form>
    <a href="{% url 'student_list' %}">Back</a>
</body>
</html>
```

---

### 🔸 `student_confirm_delete.html`

```html
<!DOCTYPE html>
<html>
<head>
    <title>Delete Student</title>
</head>
<body>
    <h2>Are you sure you want to delete {{ student.name }}?</h2>
    <form method="post">
        {% csrf_token %}
        <button type="submit">Confirm</button>
    </form>
    <a href="{% url 'student_list' %}">Cancel</a>
</body>
</html>
```

---

### ✅ STEP 9: Run the Server

```bash
python manage.py runserver
```

Visit: [http://127.0.0.1:8000/](http://127.0.0.1:8000/)
